In [286]:
import pandas as pd
import math
# price : 제품을 얻기 위해 지불하는 가격
# retail_price : 시장 또는 다른 상점/장소에서 유사한 물품에 대한 참조 가격. 
# currency_buyer : 현재 가격
# units_sold : 판매된 개수
# rating : 평균제품등급
# rating_count : 제품의 총 등급 개수
# badges_count : 제품 또는 판매자가 보유한 배지 수
# badge_local_product : 지역 제품이냐 아니냐 (1이면 지역제품)
# tags : 판매자가 붙인 꼬리표
# product_color : 제품 색
# countries_shipped_to : 이 제품이 배송되는 국가 수
# merchant_rating_count : 이 판매자의 등급 수
# merchant_rating : 상인의 등급.
# product_id : 제품 식별자
# urgency_text : 검색 결과의 일부 제품 위에 나타나는 텍스트 배너
#              - Réduction sur les achats en gros : 도매 구매 할인
#              - Quantité limitée : 제한 수량

summer_products=pd.read_csv("./data/summer-products-with-rating-and-performance_2020-08.csv")

In [287]:
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer

scaler_df=summer_products[["units_sold","merchant_rating_count","rating_count"]].astype(float)

new_df=summer_products[["price","rating","urgency_text"]]

new_df["urgency_text"]=new_df["urgency_text"].fillna("-")

urgency_text=new_df["urgency_text"].drop_duplicates().to_numpy()

encoder={}
for i,text in enumerate(urgency_text):
    encoder[text]=i

new_df["urgency_num"]=new_df["urgency_text"].apply(lambda x:encoder[x])
new_df=new_df.drop("urgency_text",axis=1)

scaler=Normalizer()
scaler.fit(scaler_df)
train_data_standardScaled = scaler.transform(scaler_df)
train_data_standardScaled= pd.DataFrame(train_data_standardScaled,columns=scaler_df.columns)
result1 = pd.concat([train_data_standardScaled,new_df],axis=1)
result1

<ipython-input-287-f3d630bfc57b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["urgency_text"]=new_df["urgency_text"].fillna("-")
<ipython-input-287-f3d630bfc57b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["urgency_num"]=new_df["urgency_text"].apply(lambda x:encoder[x])


,units_sold,merchant_rating_count,rating_count,price,rating,urgency_num
0,0.172635,0.980565,0.093223,16.00,3.76,0
1,0.728951,0.647017,0.223606,8.00,3.45,0
2,0.320716,0.946111,0.044900,8.00,3.57,0
3,0.205274,0.978416,0.023771,8.00,4.03,1
4,0.006905,0.999975,0.001381,2.72,3.10,0
...,...,...,...,...,...,...
1568,0.876625,0.466014,0.119835,6.00,4.08,1
1569,0.022542,0.999726,0.006312,2.00,3.07,0
1570,0.416735,0.875144,0.245874,5.00,3.71,1
1571,0.954983,0.296045,0.019100,13.00,2.50,1


- SVM(Support Vector Machine)\
    : 이항 분류 문제를 위해 만들졌다
    <img src="./image/SVM.png" height="400px" width="400px" >
    
    - 클래스들이 선형경계에 의해 분류될 수 있는 데이터에만 사용가능
    - 결정 경계 : 클래스들을 구분하기 위한 선
    - 서포트 벡터 : 결정 경계 근처의 가장 가까운 포인터들
    - 마진(Margin) : 결정 경계와 서포트 벡터사이의 거리
    - 파라미터
        - C : 데이터 샘픔들이 다른 클래스에 놓이는 것을 허용하는 정도 
        - kernel : 주어진 데이터를 고차원 특징 공간으로 사상(결정 경계 정의 모형)\
        (linear, plotnormaial, RBF)
        - gamma : 결정 경계의 곡률을 결정
- SVM 종류
    - SVR(Support Vector Regression)
        - 회귀에 적용하는 방법
            - svm 회귀는 제한된 마진 오류(즉, 도로 밖의 샘플)안에서 도로 안에 가능한 한 많은 샘플이 들어가도록 학습한다
            - 도로 폭은 epsilon 하이퍼파라미터로 조절
            - 마진의 범위를 넘어선 점들에 대한 error를 기준으로 model cost를 계산한다고 한다<참고>
             <img src="./image/SVR.png" height="300px" width="300px" >
    - SVC(Support Vector Classifiction)
        - 일정한 마진 오류 안에서, 두 클래스 간의 도로 폭이 가능한 한 최대가 되도록 한다
        - 마진안에 포함된 점들의 error를 기준으로 cost를 계산한다고 한다.<참고>
         <img src="./image/SVC.png" height="300px" width="300px" >
        


In [292]:
from sklearn.svm import SVR # 비선형 회귀
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


x=result1.to_numpy()[:,1:]
y=result1["price"].to_numpy()
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

scores=[]
for c in range(1,31):
    # 모델 생성 및 학습
    svr=SVR(C=c).fit(X_train,y_train)
    # train/test 예측 정확도 
    scoretrain=svr.score(X_train,y_train)
    scoretest=svr.score(X_test,y_test)
    # 평균 오차 제곱
    mse=mean_squared_error(y_test, predict_y)
    print(" C:{} \n train_score : {:.4f}, test_score: {:.4f}, mse: {:.4f}".format(c,scoretrain,scoretest,mse))


 C:1 
 train_score : 0.9902, test_score: 0.7600, mse: 31.0418
 C:2 
 train_score : 0.9976, test_score: 0.7698, mse: 31.0418
 C:3 
 train_score : 0.9986, test_score: 0.7722, mse: 31.0418
 C:4 
 train_score : 0.9992, test_score: 0.7746, mse: 31.0418
 C:5 
 train_score : 0.9994, test_score: 0.7760, mse: 31.0418
 C:6 
 train_score : 0.9996, test_score: 0.7771, mse: 31.0418
 C:7 
 train_score : 0.9997, test_score: 0.7780, mse: 31.0418
 C:8 
 train_score : 0.9997, test_score: 0.7785, mse: 31.0418
 C:9 
 train_score : 0.9998, test_score: 0.7787, mse: 31.0418
 C:10 
 train_score : 0.9998, test_score: 0.7789, mse: 31.0418
 C:11 
 train_score : 0.9998, test_score: 0.7788, mse: 31.0418
 C:12 
 train_score : 0.9998, test_score: 0.7788, mse: 31.0418
 C:13 
 train_score : 0.9998, test_score: 0.7799, mse: 31.0418
 C:14 
 train_score : 0.9998, test_score: 0.7798, mse: 31.0418
 C:15 
 train_score : 0.9998, test_score: 0.7798, mse: 31.0418
 C:16 
 train_score : 0.9998, test_score: 0.7797, mse: 31.0418
 

In [293]:
# 교차 검증을 통한 score평가
from sklearn.model_selection import cross_val_score,KFold

svr=SVR(C=30).fit(X_train,y_train)
scores=cross_val_score(svr,X_train,y_train,cv=KFold(5, random_state=10, shuffle=True))
print("5번의 cv결과 => " , scores)
print("cv 평균 =>",scores.mean())

5번의 cv결과 =>  [0.99978337 0.99977514 0.99951796 0.99978507 0.9997748 ]
cv 평균 => 0.9997272668404392


In [295]:
# GridSearchCV로 최적의 하이퍼 파라미터 찾기
from sklearn.model_selection import GridSearchCV
param={'C':[1,5,15,20,25,30],
      'gamma':[.1, .25, .5, 1]}
gf=GridSearchCV(SVR(kernel='rbf'),param, cv=5)
gf.fit(X_train,y_train)
print("최적의 하이퍼 파라미터 : ",gf.best_params_)
print("평균 예측 정확도 : ",gf.best_score_)

최적의 하이퍼 파라미터 :  {'C': 15, 'gamma': 0.1}
평균 예측 정확도 :  0.999042788466098


In [296]:
svc=SVR( C=15, gamma=.1).fit(X_train,y_train)
pred=svc.predict(X_test)
[("y_test :",round(y_test[i],2),"pred: ",round(y,4)) for i,y in enumerate(pred)]

[('y_test :', 5.65, 'pred: ', 5.5914),
 ('y_test :', 6.0, 'pred: ', 5.9779),
 ('y_test :', 6.0, 'pred: ', 5.9819),
 ('y_test :', 13.0, 'pred: ', 13.0294),
 ('y_test :', 11.0, 'pred: ', 11.101),
 ('y_test :', 2.0, 'pred: ', 2.1416),
 ('y_test :', 2.0, 'pred: ', 1.8961),
 ('y_test :', 11.0, 'pred: ', 11.0417),
 ('y_test :', 1.86, 'pred: ', 1.8057),
 ('y_test :', 5.0, 'pred: ', 4.9886),
 ('y_test :', 15.0, 'pred: ', 14.9774),
 ('y_test :', 8.0, 'pred: ', 7.9386),
 ('y_test :', 11.0, 'pred: ', 11.0283),
 ('y_test :', 12.0, 'pred: ', 12.0372),
 ('y_test :', 7.0, 'pred: ', 6.911),
 ('y_test :', 2.0, 'pred: ', 2.1117),
 ('y_test :', 13.0, 'pred: ', 13.0349),
 ('y_test :', 9.0, 'pred: ', 8.9446),
 ('y_test :', 5.0, 'pred: ', 4.9849),
 ('y_test :', 1.74, 'pred: ', 1.6729),
 ('y_test :', 4.86, 'pred: ', 4.8752),
 ('y_test :', 11.0, 'pred: ', 11.0222),
 ('y_test :', 11.0, 'pred: ', 11.0218),
 ('y_test :', 8.0, 'pred: ', 7.9153),
 ('y_test :', 8.0, 'pred: ', 7.9167),
 ('y_test :', 9.0, 'pred: ', 8

In [297]:
from sklearn.svm import LinearSVR # 선형 회귀
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

x=result1.to_numpy()[:,1:]
y=result1["price"].to_numpy()
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

scores=[]
for c in range(1,51,5):
    # 모델 생성 및 학습
    linesvr=LinearSVR(C=c).fit(X_train,y_train)
    scoretrain=svr.score(X_train,y_train)
    scoretest=svr.score(X_test,y_test)
    mse=mean_squared_error(y_test, predict_y)
    print(" C:{} \n train_score : {:.4f}, test_score: {:.4f}, mse: {:.4f}".format(c,scoretrain,scoretest,mse))

 C:1 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239
 C:6 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239
 C:11 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239
 C:16 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239
 C:21 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239
 C:26 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239
 C:31 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239
 C:36 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239
 C:41 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239
 C:46 
 train_score : 0.9326, test_score: 0.9998, mse: 30.2239


In [299]:
from sklearn.model_selection import GridSearchCV

param={'C':[1,5,15,20,25,30]}
gs=GridSearchCV(SVR(),param, cv=5)
gs.fit(X_train,y_train)
print("최적의 하이퍼 파라미터",gs.best_params_)
print("평균 예측 정확도 : ",round(gs.best_score_,4))

최적의 하이퍼 파라미터 {'C': 30}
평균 예측 정확도 :  0.9509


In [300]:
# 모델 학습
svc=LinearSVR( C=30).fit(X_train,y_train)
# 모델 예측
pred=svc.predict(X_test)
[("y_test :",round(y_test[i],2),"pred: ",round(y,4)) for i,y in enumerate(pred)]

[('y_test :', 5.69, 'pred: ', 5.69),
 ('y_test :', 14.0, 'pred: ', 14.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 14.0, 'pred: ', 14.0),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 3.83, 'pred: ', 3.83),
 ('y_test :', 2.71, 'pred: ', 2.71),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 11.0, 'pred: ', 11.0),
 ('y_test :', 14.0, 'pred: ', 14.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 12.0, 'pred: ', 12.0),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 6.0, 'pred: ', 6.0),
 ('y_test :', 5.65, 'pred: ', 5.65),
 ('y_test :', 8.0, 'pred: ', 8.0),
 ('y_test :', 6.0, 'pred: ', 6.0),
 ('y_test :', 2.71, 'pred: ', 2.71),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 18.0, 'pred: ', 18.0),
 ('y_test :', 8.0, 'pred: ', 8.0),
 ('y_test :', 16.0, 'pred: ', 16.0),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 2.65, 'pred: ', 2.65),
 ('y_test :', 6.0, 'pred: ', 

In [302]:
# DecisionTreeRegressor(회귀 결정)
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# 모델 생성 및 학습
regr_tree=DecisionTreeRegressor(max_depth =3,random_state =11).fit(X_train,y_train )

param={'max_depth':[1,3,5,7,9,11,13,15],"min_samples_split":[12,15]}
gs=GridSearchCV(regr_tree,param,cv=5,verbose=1)
gs.fit(X_train,y_train)
print("최적의 하이퍼 파라미터:" ,gs.best_params_)
print("평균 정확도 :",round(gs.best_score_,4))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
최적의 하이퍼 파라미터: {'max_depth': 9, 'min_samples_split': 12}
평균 정확도 : 0.9719


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished


In [303]:
# 모델 학습
regr_tree=DecisionTreeRegressor(max_depth= 9, min_samples_split= 12).fit(X_train,y_train)

# 모델 예측
pred=regr_tree.predict(X_test)
[("y_test :",round(y_test[i],2),"pred: ",round(y,4)) for i,y in enumerate(pred)]

[('y_test :', 5.69, 'pred: ', 5.6783),
 ('y_test :', 14.0, 'pred: ', 14.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 14.0, 'pred: ', 14.0),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 3.83, 'pred: ', 3.8582),
 ('y_test :', 2.71, 'pred: ', 2.7111),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 11.0, 'pred: ', 11.0),
 ('y_test :', 14.0, 'pred: ', 14.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 12.0, 'pred: ', 12.0),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 6.0, 'pred: ', 6.0),
 ('y_test :', 5.65, 'pred: ', 5.65),
 ('y_test :', 8.0, 'pred: ', 8.0),
 ('y_test :', 6.0, 'pred: ', 6.0),
 ('y_test :', 2.71, 'pred: ', 2.7111),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 7.0, 'pred: ', 7.0),
 ('y_test :', 18.0, 'pred: ', 17.8889),
 ('y_test :', 8.0, 'pred: ', 8.0),
 ('y_test :', 16.0, 'pred: ', 16.0),
 ('y_test :', 9.0, 'pred: ', 9.0),
 ('y_test :', 2.65, 'pred: ', 2.664),
 ('y_test :', 6.0

In [304]:
# 모델 예측
print("예측 정확도 : ",round(mean_squared_error(pred,y_test),4))

예측 정확도 :  0.0319


In [305]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param={"max_depth":[1,3,5,7,9], "min_samples_leaf":[5,9,11],"min_samples_split":[7,11]}
rfr=RandomForestRegressor(random_state=11,n_jobs=-1,max_depth=1)
gs=GridSearchCV(rfr,param, cv=5,n_jobs=-1)
gs.fit(X_train,y_train)
print("최적의 하이퍼 파라미터",gs.best_params_)
print("평균 예측 정확도 : ",round(gs.best_score_,4))

최적의 하이퍼 파라미터 {'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 7}
평균 예측 정확도 :  0.9726


In [306]:
# 모델 학습
regr_tree=RandomForestRegressor(max_depth= 7,min_samples_leaf=5, min_samples_split= 7).fit(X_train,y_train)

# 모델 예측
pred=regr_tree.predict(X_test)
#[("y_test :",round(y_test[i],2),"pred: ",round(y,4)) for i,y in enumerate(pred)]
print("예측 정확도 : ",round(mean_squared_error(pred,y_test),4))

예측 정확도 :  0.0355


In [ ]:
#  https://analysis-flood.tistory.com/96 
# https://bskyvision.com/163 
# https://chancoding.tistory.com/67
# https://todayisbetterthanyesterday.tistory.com/33
# http://hleecaster.com/ml-svm-concept/#comment-785 
# https://jeongmin-lee.tistory.com/87